In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import KFold
from surprise import KNNBasic,KNNWithMeans
from surprise import accuracy
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import train_test_split
from collections import defaultdict
import io
from surprise.model_selection import GridSearchCV
import csv

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

data_old = pd.read_csv('C:/Users/arshi/Desktop/ArshiyaUSB/Fall2018/256/Project/CMPE256-Airbnb/Module1/generated_rating.csv')

#data_old = pd.read_csv('../input/reviews/generated_rating.csv')

data_old.head()
data_old.nunique()
print("===========")
df = data_old[['reviewer_id','listing_id','Rating' ]]
df.head()
print(df.nunique())

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['reviewer_id', 'listing_id', 'Rating']], reader)

# define a cross-validation iterator
kf = KFold(n_splits=7)
algo = SVD(n_factors=500, n_epochs=5, lr_all=0.1)

for trainset, testset in kf.split(data):
    # train and test algorithm.
    algo.fit(trainset)
    
    predictions = algo.test(testset)
    
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)
print(len(predictions))

# Retrieve the trainset.
print("After taking full train test")
#trainset = data.build_full_trainset()
#testset = trainset.build_anti_testset()
raw_ratings = data.raw_ratings

# shuffle ratings if you want
#random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(.7 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# Select your best algo with grid search.
print('Grid Search...')

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
grid_search.fit(data)
print('best RMSE score')
print(grid_search.best_score['rmse'])

print('combination of parameters that gave the best RMSE score')
print(grid_search.best_params['rmse'])
algo = grid_search.best_estimator['rmse']

# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
testset= trainset.build_anti_testset()
predictions = algo.test(testset)
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions,verbose=True)
print('len(predictions)')
print(len(predictions))
 
'''
#trainset, testset = train_test_split(data, test_size=0.85)

algo.fit(trainset)
# Than predict ratings for all pairs (u, i) that are NOT in the training set.

predictions = algo.test(testset)
print(len(predictions))
# Compute and print Root Mean Squared Error
accuracy.rmse(predictions, verbose=True)   
'''
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]    
    return top_recs
print('getting top 3 recommendations')

top3_recommendations = get_top3_recommendations(predictions,3)

print('len(top3_recommendations)')
print(len(top3_recommendations))


dfo = pd.DataFrame(columns=['UserId', 'Recommended Listing,Rating'])
i=0;
for uid, user_ratings in top3_recommendations.items():
    print(uid, top3_recommendations[uid])
    row = [uid, top3_recommendations[uid]]
    dfo.loc[i] = row
    i=i+1
    if(i==10):
        break

dfo.to_csv('sub.csv', index = False)

print("done")     
        



reviewer_id    9969
listing_id     6983
Rating            5
dtype: int64
RMSE: 1.2113
RMSE: 1.2013
RMSE: 1.2154
RMSE: 1.2194
RMSE: 1.1916
RMSE: 1.1766
RMSE: 1.2000
1427
After taking full train test
Grid Search...
best RMSE score
1.20061847361
combination of parameters that gave the best RMSE score
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}
Biased accuracy on A,   RMSE: 0.1010
len(predictions)
37386974
getting top 3 recommendations
len(top3_recommendations)
6983
36894430 [(14369923, 4.2801860113863981), (1185644, 4.2642601679730081), (1073832, 4.2241796871781121)]
9765927 [(1881801, 4.2172184632173408), (2134047, 4.1972767775000435), (10423083, 4.1815109399793133)]
12936998 [(4501541, 4.2277572183029024), (1737891, 4.218068178636087), (2214323, 4.1993199628622184)]
345461 [(12028699, 4.2869922916636485), (13292687, 4.2195689032446637), (8564421, 4.2035988661923973)]
6171559 [(2461540, 4.2375517962309734), (19209710, 4.2145062136136886), (14665418, 4.2080897381983409)]
3414482 [(72